In [1]:
import ccxt
import pandas as pd
import ta

In [2]:
exchange = ccxt.binance({
    'enableRateLimit': True,  # important to avoid ban
})


In [3]:
markets = exchange.load_markets()

In [4]:
usdt_pairs = []

for symbol, market in markets.items():
    if (
        symbol.endswith('/USDT') and
        market['spot'] and
        market['active']
    ):
        usdt_pairs.append(symbol)

print(f"Total active USDT spot pairs: {len(usdt_pairs)}")
print(usdt_pairs[:10])  # show first 10


Total active USDT spot pairs: 441
['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'NEO/USDT', 'LTC/USDT', 'QTUM/USDT', 'ADA/USDT', 'XRP/USDT', 'TUSD/USDT', 'IOTA/USDT']


In [5]:
filtered_pairs = [
    symbol for symbol in usdt_pairs
    if not any(x in symbol for x in ['UP/', 'DOWN/', 'BULL/', 'BEAR/'])
]

print(f"Clean USDT pairs: {len(filtered_pairs)}")


Clean USDT pairs: 439


In [6]:
def get_ohlcv(symbol, timeframe='15m', limit=200):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    
    df = pd.DataFrame(ohlcv, columns=[
        'timestamp','open','high','low','close','volume'
    ])
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df


In [7]:
import pandas as pd
import numpy as np

def add_rsi_tradingview(df, length=14, source_col='close'):
    """
    TradingView / Pine Script RSI implementation using RMA (Wilder)
    """

    # 1. Price change (ta.change)
    change = df[source_col].diff()

    # 2. Up & Down series
    up = change.clip(lower=0)
    down = -change.clip(upper=0)

    # 3. Wilder's Moving Average (RMA)
    alpha = 1 / length
    avg_up = up.ewm(alpha=alpha, adjust=False).mean()
    avg_down = down.ewm(alpha=alpha, adjust=False).mean()

    # 4. RSI calculation
    rs = avg_up / avg_down
    rsi = np.where(
        avg_down == 0, 100,
        np.where(avg_up == 0, 0, 100 - (100 / (1 + rs)))
    )

    df['rsi'] = rsi
    return df


In [8]:
def add_stochastic_tv(df, periodK=14, smoothK=3, periodD=3):
    
    # Raw stochastic
    low_min = df['low'].rolling(periodK).min()
    high_max = df['high'].rolling(periodK).max()
    
    stoch = 100 * (df['close'] - low_min) / (high_max - low_min)
    
    # Smooth K
    k = stoch.rolling(smoothK).mean()
    
    # Smooth D
    d = k.rolling(periodD).mean()
    
    df['stoch_k'] = k
    df['stoch_d'] = d
    
    return df

In [9]:
def add_stoch_rsi_tv(df, lengthRSI=14, lengthStoch=14, smoothK=3, smoothD=3):

    # --- TRUE TradingView RSI (RMA based)
    delta = df['close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    alpha = 1 / lengthRSI
    avg_gain = gain.ewm(alpha=alpha, adjust=False).mean()
    avg_loss = loss.ewm(alpha=alpha, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # --- TRUE Stochastic of RSI
    rsi_min = rsi.rolling(lengthStoch).min()
    rsi_max = rsi.rolling(lengthStoch).max()

    stoch = 100 * (rsi - rsi_min) / (rsi_max - rsi_min)

    # --- Smooth K and D
    k = stoch.rolling(smoothK).mean()
    d = k.rolling(smoothD).mean()

    df['stoch_rsi_k'] = k
    df['stoch_rsi_d'] = d

    return df

In [10]:
df = get_ohlcv('ADA/USDT', '1d', 200)
df = add_rsi_tradingview(df)
df = add_stochastic_tv(df)
df = add_stoch_rsi_tv(df, 14, 14, 3, 3)

In [11]:
last = df.iloc[-1]

print("Symbol: ADA/USDT")
print("RSI:", last['rsi'])
print("stochastic RSI K", last['stoch_rsi_k']) #Required for stochastic(14, 14, 3, 3) strategy
print("stochastic %K(14, 3, 3)", last['stoch_d']) # required for stochastic%K(14,3, 3) strategy

Symbol: ADA/USDT
RSI: 35.00539446246863
stochastic RSI K 82.35689789596684
stochastic %K(14, 3, 3) 31.887721163517202


In [12]:
def check_signal(df):
    last = df.iloc[-1]

    rsi = last["rsi"]
    stoch_rsi = last["stoch_rsi_k"]   # stochastic(14,14,3,3)
    stoch_k = last["stoch_d"]         # stochastic %K(14,3,3)

    # === Condition 1: Oversold ===
    oversold = (
        rsi < 45 and
        stoch_rsi < 45 and
        stoch_k < 45
    )

    # === Condition 2: Momentum shift ===
    momentum = stoch_k > stoch_rsi

    return oversold and momentum


In [13]:
timeframes = ['15m', '1h', '4h', '1d']

def analyze_symbol(symbol):
    agreeing_timeframes = []

    for tf in timeframes:
        try:
            df = get_ohlcv(symbol, tf, 500)

            df = df.iloc[:-1]  # remove live candle

            df = add_rsi_tradingview(df)
            df = add_stochastic_tv(df, 14, 3, 3)
            df = add_stoch_rsi_tv(df, 14, 14, 3, 3)

            if check_signal(df):
                agreeing_timeframes.append(tf)

        except Exception as e:
            print(f"Error {symbol} {tf}: {e}")

    return agreeing_timeframes


In [14]:
def scan_market(pairs):
    results = {}

    for symbol in pairs:
        print(f"Scanning {symbol}...")
        agreeing = analyze_symbol(symbol)

        if len(agreeing) >= 2:  # require at least 2 TFs agreement
            results[symbol] = agreeing

    return results


In [15]:
signals = scan_market(filtered_pairs)

print("\n=== FINAL SIGNALS ===")
for pair, tfs in signals.items():
    print(f"{pair} → {tfs}")


Scanning BTC/USDT...
Scanning ETH/USDT...
Scanning BNB/USDT...
Scanning NEO/USDT...
Scanning LTC/USDT...
Scanning QTUM/USDT...
Scanning ADA/USDT...
Scanning XRP/USDT...
Scanning TUSD/USDT...
Scanning IOTA/USDT...
Scanning XLM/USDT...
Scanning ONT/USDT...
Scanning TRX/USDT...
Scanning ETC/USDT...
Scanning ICX/USDT...
Scanning VET/USDT...
Scanning USDC/USDT...
Scanning LINK/USDT...
Scanning ONG/USDT...
Scanning HOT/USDT...
Scanning ZIL/USDT...
Scanning ZRX/USDT...
Scanning FET/USDT...
Scanning BAT/USDT...
Scanning ZEC/USDT...
Scanning IOST/USDT...
Scanning CELR/USDT...
Scanning DASH/USDT...
Scanning THETA/USDT...
Scanning ENJ/USDT...
Scanning ATOM/USDT...
Scanning TFUEL/USDT...
Scanning ONE/USDT...
Scanning ALGO/USDT...
Scanning DOGE/USDT...
Scanning DUSK/USDT...
Scanning ANKR/USDT...
Scanning WIN/USDT...
Scanning COS/USDT...
Scanning MTL/USDT...
Scanning DENT/USDT...
Scanning WAN/USDT...
Scanning FUN/USDT...
Scanning CVC/USDT...
Scanning CHZ/USDT...
Scanning BAND/USDT...
Scanning XTZ/US

In [16]:
print("\n=== FINAL SIGNALS ===")
for pair, tfs in signals.items():
    print(f"{pair} → {tfs}")


=== FINAL SIGNALS ===
BNB/USDT → ['15m', '1h']
ROSE/USDT → ['4h', '1d']
RIF/USDT → ['15m', '4h']
CAKE/USDT → ['1h', '4h']
BICO/USDT → ['15m', '1h']
GUN/USDT → ['1h', '4h']
HEMI/USDT → ['1h', '4h']
